Main Imports

In [1]:
from typing import List
from Dataset.CustomDataset import AgeGroupAndAgeDataset, StandardDataset, AgeGroupAndAgeDatasetKL
from Dataset.CustomDataLoaders import CustomDataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from Utils import AAR, CSVUtils, AgeConversion
from Utils.Validator import Validator
import torch
import torch.nn as nn
import torch.nn.functional as F

c:\Users\geral\miniconda3\envs\AV\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Caricamento del dataframe
df = CSVUtils.get_df_from_csv("./training_caip_contest.csv", "./training_caip_contest/")

#Suddivisione del dataframe in 3 age groups
_, d = CSVUtils.get_df_with_age_subdivision(df, 3)

In [3]:
#Splitting tra Train e Validation set
df_train, df_val = train_test_split(df, test_size=0.25, random_state=42)
#Aggiornamento degli indici per pandas
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

from PIL import Image
import numpy as np
from torchvision import transforms
import torch

transform_func = transforms.Compose([
    transforms.Resize(224),
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
    transforms.RandomGrayscale(),
])

transform_func_val = transforms.Compose([
    transforms.Resize(224),
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

#Implementazione di un Dataset utilizzando "CustomDataset" per l'architettura con Film
cd_train = AgeGroupAndAgeDatasetKL(df_train, path_col="path", label_col="age", label_function="CAE", 
                                 label_map=d, label_map_n_classes=3, transform_func=transform_func)

#Implementazione di un Dataset che adatta le label all'utilizzo che vogliamo farne (in questo caso CAE = Cathegorical)
cd_val = StandardDataset(df_val, path_col="path", label_col="age", label_function="CAE", transform_func=transform_func_val)

#Dato che lo split potrebbe non prendere sample di determinate classi facciamo il set del numero di classi
cd_train.set_n_classes(101)

#Loader che, conoscendo la grandezza del dataset, farà lo shuffle dei campioni e crea i batch
dm_train, dm_val = CustomDataLoader(cd_train), CustomDataLoader(cd_val)
#Utilizziamo il dataloader che crea batch bilanciati implementato in CustomDataLoaders
# In generale non usare questo ma l'unbalanced dato che non è usato in nessun doc
dl_train = dm_train.get_unbalanced_dataloader(batch_size=128 ,shuffle=True, num_workers=6, prefetch_factor=4)


In [4]:
import torchvision.models as models

# Initialize the ResNet18 model with pre-trained parameters on ImageNet
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
# Change the dimension of the fully connected layer to K
K = 101
model.fc = nn.Linear(model.fc.in_features, K)

In [5]:
#Validator che si occuperà di tenere in considerazione le metriche da massimizzare per il contest
validator = Validator(cd_val, AgeConversion.ArgMaxAge, 128)

In [6]:
import torchvision.transforms as transforms

# Define the transformation that resizes the image and flips it horizontally with a probability of 0.5
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5)
])

In [7]:
# Define the KL divergence loss
def kl_divergence_loss(pred, target):
    # Calculate the KL divergence between the label distribution and the predicted age distribution
    loss = nn.KLDivLoss(reduction='batchmean')(pred.log(), target)
    return loss


# Define the L1 loss
def l1_loss(pred, target):
    # Calculate the L1 loss between the predicted age and the ground truth label
    loss = nn.L1Loss()(pred, target)
    return loss

# Combine the KL divergence loss and L1 loss
def combined_loss(pred, target):
    return kl_divergence_loss(pred, target) + l1_loss(pred, target)


In [8]:
import torch.optim as optim

best_val_aar = -1
# Define the optimization algorithm
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9)

# Define the learning rate scheduler
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20, 40, 60], gamma=0.1)

model.to("cuda")
# Train the model for a total of 75 epochs
for epoch in range(75):
    # Train the model on the training data
    model.train()
    with tqdm(dl_train, unit=" batch") as tepoch:
        for input, target in tepoch:
            input = input.to("cuda")
            target = target[-1].to("cuda")
            # Clear the gradients
            optimizer.zero_grad()

            # Forward pass
            output = F.softmax(model(input), dim=-1)

            # Calculate the loss
            loss = combined_loss(output, target)

            # Backward pass
            loss.backward()

            # Update the model parameters
            optimizer.step()

            tepoch.set_postfix(loss=loss.detach().cpu().numpy()) 
        
    # Decrease the learning rate
    scheduler.step()

    def forward_function(x):
        return F.softmax(model(x), dim=-1)

    val_aar, val_aar_old = validator.validate(forward_function)
    print(val_aar, val_aar_old)

    if val_aar > best_val_aar:
        best_val_aar = val_aar
        torch.save(model.state_dict(), "./model_age_baseline.pt")
        print("Model saved")


  1%|          | 29/3370 [01:14<2:23:06,  2.57s/ batch, loss=2.585818169013351] 


KeyboardInterrupt: 